# Evaluation

1. autogluon top10 model evaluation

In [ ]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import utils.eda_utils as eda_utils 
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score
import numpy as np

In [ ]:
test_df = pd.read_csv('data/241005_Rockhead_Seoul_test(before_aumentation).csv')

In [ ]:
test_df['ground_name'] = test_df['ground_name'].astype('category')
test_df['rock_type'] = test_df['rock_type'].astype('category')

test = TabularDataset(test_df.drop(['depth_start', 'dis_bin'], axis=1))
y_test = test_df['depth_start'].values.tolist()

In [ ]:
predictor = TabularPredictor.load('path_to_predictor')
ld_board = predictor.leaderboard(test, silent=True)

In [ ]:
evaluation_df = pd.Dataframe(columns=['model', 'bin', 'mae', 'mse', 'rmse', 'r2', 'adjusted_r2'])

for i in range(10):
        
    for bin in test_df['dis_bin'].unique():

        bin_df = test_df[test_df['dis_bin'] == bin]
                                
        test_bin = TabularDataset(bin_df.drop(['depth_start', 'dis_bin'], axis=1))
        y_test_bin = bin_df['depth_start'].values.tolist()

        pred_y = predictor.predict(test_bin, ld_board.model[i])

        mae = mean_absolute_error(y_test_bin, pred_y)
        mse = mean_squared_error(y_test_bin, pred_y)
        rmse = root_mean_squared_error(y_test_bin, pred_y)
        r2 = r2_score(y_test_bin, pred_y)

        n = len(bin_df)
        k = 6
        adjusted_r2 = (1-r2)(n-1)/(n-k-1)

        evaluation_df.append({'model':ld_board.model[i],'bin':bin, 'mae':mae, 'mse':mse, 'rmse':rmse, 'r2':r2, 'adjusted_r2':adjusted_r2}, ignore_index=True)
    
    evaluation_df[evaluation_df['model']==ld_board.model[i]].plot.bar(x='bin', y='adjusted_r2')

evaluation_df.to_csv('data/241025_Rockhead_Seoul_Evaluation(before_aumentation).csv', index=False)